# Extract train-dev-test csv datasets from BigQuery using dataframe

In [8]:
import google.datalab.bigquery as bq
import pandas as pd
import numpy as np
import seaborn as sns
import shutil

In [66]:

def sample_between(a, b):
  basequeryX = """
  SELECT 
    answer_count, comment_count, favorite_count,  score, view_count,
    TIMESTAMP_DIFF(last_activity_date, creation_date, DAY) as days_posted,
    IF(accepted_answer_id IS NULL , 0, 1) as accepted,
    (IF(answer_count IS NULL , 0, answer_count*2) 
    + IF(comment_count IS NULL , 0, comment_count)  
    + IF(favorite_count IS NULL , 0, favorite_count)  
    + IF(score IS NULL , 0, score)  
    + IF(view_count IS NULL , 0, view_count*2) ) 
    as total_score
  FROM 
    `bigquery-public-data.stackoverflow.posts_questions`
  """
  basequery = """
  SELECT 
    answer_count, comment_count, favorite_count,  score, view_count,
    TIMESTAMP_DIFF(last_activity_date, creation_date, DAY) as days_posted,
    IF(accepted_answer_id IS NULL , 0, 1) as accepted
  FROM 
    `bigquery-public-data.stackoverflow.posts_questions`
  """
  sampler = "WHERE MOD(ABS(FARM_FINGERPRINT(CAST(id as STRING))), EVERY_N * 100) <= 10"
  sampler2 = "AND {0} >= {1}\n AND {0} < {2}".format(
           "MOD(ABS(FARM_FINGERPRINT(CAST(id AS STRING))), EVERY_N * 100) * {}".format(10),
           a, b
          )
  return "{}\n{}\n{}".format(basequery, sampler, sampler2)


def create_query(phase, EVERY_N):
  """Phase: train (70%) valid (15%) or test (15%)"""
  query = ""
  if phase == 'train':
    query = sample_between(0,70)
  elif phase == 'valid':
    query = sample_between(70,85)
  else:
    query = sample_between(85,100)
  return query.replace("EVERY_N", str(EVERY_N))

#print(create_query('train', 100))
#(answer_count - AVG(answer_count)) / STDDEV_POP(answer_count)  as answer_count,

In [74]:
def to_csv(df, filename):
  outdf = df.copy(deep = False)
  #outdf.loc[:, 'key'] = np.arange(0, len(outdf)) # rownumber as key
  # Reorder columns so that target is first column
  cols = outdf.columns.tolist()
  #print(cols)
  cols.remove('accepted')
  cols.insert(0, 'accepted')
  #print(cols)
  outdf = outdf[cols]
  print(outdf[0,0])
  #Normalizing input columns 
  #outdf[]
  outdf.to_csv(filename,  header = True, index_label = False, index = False)
  print("Wrote {} to {}".format(len(outdf), filename))

In [75]:

for phase in ['train', 'valid', 'test']:
  query = create_query(phase, 100)
  df = bq.Query(query).execute().result().to_dataframe()
  to_csv(df, 'stackoverflow-{}.csv'.format(phase))

KeyError: (0, 0)

In [69]:
!ls -l *.csv

-rw-r--r-- 1 root root  27815 May 15 04:32 stackoverflow-test.csv
-rw-r--r-- 1 root root 194705 May 15 04:32 stackoverflow-train.csv
-rw-r--r-- 1 root root  55339 May 15 04:32 stackoverflow-valid.csv


In [70]:
%bash
head stackoverflow-train.csv

accepted,answer_count,comment_count,favorite_count,score,view_count,days_posted
0,7,1,2.0,11,20478,451
1,8,4,2.0,8,6964,1147
0,11,4,,0,870,2170
0,8,4,2.0,14,3269,0
1,8,2,101.0,210,214397,3013
0,7,1,9.0,17,6580,2078
1,7,0,1.0,0,10191,3066
1,8,4,,2,90,0
1,7,0,11.0,14,15595,2003


# tf.estimator modeling

In [13]:
# Ensure that we have TensorFlow 1.13.1 installed.
!pip3 freeze | grep tensorflow==1.13.1 || pip3 install tensorflow==1.13.1

    100% |████████████████████████████████| 92.5MB 366kB/s eta 0:00:01  3% |█                               | 3.0MB 23.0MB/s eta 0:00:04    8% |██▉                             | 8.1MB 33.8MB/s eta 0:00:03    10% |███▎                            | 9.6MB 35.6MB/s eta 0:00:03    12% |███▉                            | 11.2MB 35.4MB/s eta 0:00:03    15% |████▉                           | 14.1MB 30.1MB/s eta 0:00:03    16% |█████▎                          | 15.4MB 28.8MB/s eta 0:00:03    17% |█████▊                          | 16.6MB 30.8MB/s eta 0:00:03    19% |██████▏                         | 17.9MB 29.9MB/s eta 0:00:03    27% |████████▉                       | 25.6MB 27.5MB/s eta 0:00:03    38% |████████████▍                   | 35.8MB 18.4MB/s eta 0:00:04    40% |████████████▉                   | 37.1MB 15.3MB/s eta 0:00:04    44% |██████████████▍                 | 41.5MB 23.2MB/s eta 0:00:03    49% |███████████████▉                | 45.8MB 22.5MB/s eta 0:00:03    51% |████████████████▌ 

In [45]:
import tensorflow as tf
import pandas as pd
import shutil

print(tf.__version__)

1.13.1


In [46]:
!ls -l *.csv

-rw-r--r-- 1 root root  27815 May 15 04:04 stackoverflow-test.csv
-rw-r--r-- 1 root root 194705 May 15 04:04 stackoverflow-train.csv
-rw-r--r-- 1 root root  55339 May 15 04:04 stackoverflow-valid.csv


In [47]:
df_train = pd.read_csv(filepath_or_buffer = "./stackoverflow-train.csv")
df_valid = pd.read_csv(filepath_or_buffer = "./stackoverflow-valid.csv")
df_test = pd.read_csv(filepath_or_buffer = "./stackoverflow-test.csv")

CSV_COLUMNNAMES = list(df_train) # CSV_COLUMNNAMES = df_train.columns.tolist()
print(CSV_COLUMNNAMES)

FEATURE_NAMES = CSV_COLUMNNAMES[1:]
LABEL_NAME = CSV_COLUMNNAMES[0]

['accepted', 'answer_count', 'comment_count', 'favorite_count', 'score', 'view_count', 'days_posted']


In [48]:
featcols = [ tf.feature_column.numeric_column(feat) for feat in  FEATURE_NAMES ]
#print(featcols)

In [49]:
def pandas_input_3(df, phase, batch_size = 128):
  dataset = tf.data.Dataset.from_tensor_slices(tensors = ( dict(df[FEATURE_NAMES]), df[LABEL_NAME] )  )
  if(phase == 'train'):
    #return tf.estimator.inputs.pandas_input_fn(x = dict(df[FEATURE_NAMES]), y= df[LABEL_NAME], batch_size = batch_size, num_epochs = 10, shuffle = True, queue_capacity = 1000)
    dataset = dataset.shuffle(buffer_size = 1000).repeat(count=None).batch(batch_size = batch_size)
    return dataset
  else:
    #return tf.estimator.inputs.pandas_input_fn(x = dict(df[FEATURE_NAMES]), y= df[LABEL_NAME], batch_size = batch_size)
    dataset = dataset.batch(batch_size = batch_size)
    return dataset

In [50]:
%%time
OUTDIR = "stack_trained"

tf.logging.set_verbosity(tf.logging.INFO)
shutil.rmtree(path = OUTDIR, ignore_errors = True)

model = tf.estimator.DNNClassifier(
    hidden_units = [10, 10],  # specify neural architecture
    feature_columns = featcols,
    n_classes=2,
    #optimizer = tf.train.AdamOptimizer(learning_rate=0.001),
    model_dir = OUTDIR,
    config = tf.estimator.RunConfig(tf_random_seed = 1)  
  )

model.train(
    input_fn = lambda : pandas_input_3(df = df_train, phase = 'train'),
    steps = 500
  )

INFO:tensorflow:Using config: {'_log_step_count_steps': 100, '_global_id_in_cluster': 0, '_save_checkpoints_secs': 600, '_is_chief': True, '_tf_random_seed': 1, '_device_fn': None, '_num_worker_replicas': 1, '_keep_checkpoint_every_n_hours': 10000, '_save_summary_steps': 100, '_num_ps_replicas': 0, '_protocol': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_service': None, '_evaluation_master': '', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fb4625548d0>, '_keep_checkpoint_max': 5, '_save_checkpoints_steps': None, '_task_type': 'worker', '_experimental_distribute': None, '_model_dir': 'stack_trained', '_eval_distribute': None, '_task_id': 0, '_train_distribute': None, '_master': ''}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_i

InvalidArgumentError: Nan in summary histogram for: dnn/dnn/hiddenlayer_0/activation
	 [[node dnn/dnn/hiddenlayer_0/activation (defined at /usr/local/envs/py3env/lib/python3.5/site-packages/tensorflow_estimator/python/estimator/canned/dnn.py:50) ]]

Caused by op 'dnn/dnn/hiddenlayer_0/activation', defined at:
  File "/usr/local/envs/py3env/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/local/envs/py3env/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/envs/py3env/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/usr/local/envs/py3env/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/envs/py3env/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 486, in start
    self.io_loop.start()
  File "/usr/local/envs/py3env/lib/python3.5/site-packages/tornado/io